In [ ]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES), and is copyright (c) 2018-2022
# by the software owners: The Regents of the University of California, through
# Lawrence Berkeley National Laboratory,  National Technology & Engineering
# Solutions of Sandia, LLC, Carnegie Mellon University, West Virginia University
# Research Corporation, et al.  All rights reserved.
#
# Please see the files COPYRIGHT.md and LICENSE.md for full copyright and
# license information.
###############################################################################

# Design flowsheet for the SOEC mode of a reversible Solid Oxide Cell

### Process Flow Diagram

In [ ]:
from IPython.core.display import SVG

SVG(filename="rsofc_soec_mode_PFD.svg")

## 1. Introduction
The figure above shows the process flow diagram of the rSOC (reversible solid oxide cell) plant in SOEC (solid oxide electrolysis cell) mode. The plant consists of five major sections: the ASU (air separation unit), the SOEC power island, the CPU (CO2 purification unit), the HRSG (heat recovery steam generator) and steam cycle.

In the SOEC mode, the rSOC operates by using steam as fuel to produce Hydrogen on its fuel side (cathode side), while air is used as sweep gas to remove diffused oxygen from its air side (anode side). To provide air to the air side of the rSOC, ambient air is compressed in an air blower and preheated via two preheaters to reach the required temperature to sweep through the air side. The oxygen rich air stream which exits the air side of the rSOC is used to preheat the fresh air feed in the first air preheater, subsequently it is split in a splitter and used to preheat oxygen from the ASU in the oxygen preheater and natural gas feed in the natural gas preheater. Finally, the oxygen rich air streams from both the oxygen and natural gas preheaters are sent to the HRSG where they are further cooled before release to the atmosphere.

This flowsheet example provides an off-design model for the rSOC in SOEC mode for hydrogen production up to **5 kg/s** with 98% CO2 capture.

## 2. Import Required Modules
- **pyomo environment**. The pyomo environment
- **rsofc_soec_flowsheet**. Contains functions for the work flow of building and solving the flowsheet. See the rsofc_soec_flowsheet.py file for more details.
- **rsofc_costing**. Contains costing methods for the soec and sofc operating modes of the reversible sofc. See the rsofc_costing.py file for more details.


In [ ]:
import pyomo.environ as pyo
import rsofc_soec_flowsheet as rsoec
import rsofc_costing as rsofc_cost

## 3. Build and initialize the flowsheet
- A pyomo concrete model is built
- The **get_model** function in the **rsofc_soec_flowsheet.py** file takes the concrete model as an argument then sets up a flowsheet called **m.soec_fs**, and all the required unit models to it. Finally, the completed flowsheet is initialized. The concrete model and solver options are then returned.

In [ ]:
m = pyo.ConcreteModel()
m, solver = rsoec.get_model(m)

## 4. Base case optimization example
An example of optimizing the variable operating costs of the model at its base case production of 5 kg/s using the **base_case_optimization** function is shown below. The model also has functions to run a base case simulation (**base_case_simulation**) 
and optimize the variable operating cost of the process over the production range of 1 to 5 kg/s (**optimize_model**)

In [ ]:
rsoec.tags_inputs_opt_vars(
    m.soec_fs
)  # this function adds tags for the optimization variables
rsofc_cost.get_rsofc_soec_variable_OM_costing(m.soec_fs)
rsoec.base_case_optimization(m, solver)

## 5. Results

### a. Input and optimization variables

In [ ]:
rsoec.display_input_tags(m.soec_fs)

### b. Stream tables

In [ ]:
stream_table = rsoec.stream_tables(m.soec_fs)
stream_table = stream_table.fillna("")  # replace NaN with blank
display(stream_table)

### c. Print results table

In [ ]:
result_variables = rsoec.initialize_results(m.soec_fs)
results_table = rsoec.results_table_dataframe(result_variables)
print("========================== Overall Results ========================")
print()
print(results_table)